In [ ]:
#단순 최다 빈도 명사
# word_cnt = Counter(nouns).most_common()
# total_df = pd.DataFrame(word_cnt, columns = ['단어', '빈도수'])
# total_df.head(10)

In [22]:
import json
import csv
import pandas as pd
from konlpy.tag import Komoran
from konlpy.corpus import kolaw
from konlpy.tag import Okt
#import nltk
#from nltk import Text
from collections import Counter
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from future.utils import iteritems
from collections import Counter
from collections import defaultdict
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
data = pd.read_csv('루리웹_엑셀확인용.csv')
data.values
values = "".join(str(i) for i in data.values)

#okt = Okt()
komoran = Komoran() 

#total_morphs = okt.morphs(values)
total_morphs = komoran.morphs(values)

print("전체 형태소 개수: ")
print(len(total_morphs))

#nouns = okt.nouns(values)
nouns = komoran.nouns(values)
print("전체 명사 개수: ")
print(len(nouns))

nouns = [noun for noun in nouns if len(noun) >= 2] #3글자 이상의 키워드를 뽑는 것도 고려해 볼 것

bool_words = ['아니', '아서', '아도', '진짜', '생각', '라고', '지만', '정도', '만나', '제가', '지금', '거지', '느낀','새끼'  
             '하나', '보이', '얘기', '마음', '우리', '사람', '가지', '생기', '잘못', '때문', '이랑', '문제', '확찢','보고']

#unique_Nouns = set(nouns)
for word in nouns:
    if word in bool_words:
        while word in nouns: 
            nouns.remove(word)


전체 형태소 개수: 
29446
전체 명사 개수: 
9871


In [24]:
data = pd.read_csv('루리웹_엑셀확인용.csv')
corpus = [' '.join(i[0].split(' ')[1:]) for i in data.values]

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(corpus):
    print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

[('김경수', 0.5), ('차기', 0.5), ('경남지사', 0.5), ('지지율', 0.5), ('위', 0.0)]
[('의원님', 0.5773502691896257), ('인스타', 0.5773502691896257), ('커여움', 0.5773502691896257)]
[('디시의', 0.5), ('멍청이들', 0.5), ('낚기', 0.5), ('대성공', 0.5)]
[('우리', 0.41392529509784665), ('민주당이', 0.37858849743594003), ('언제부터', 0.41392529509784665), ('부자들을', 0.41392529509784665), ('위한', 0.41392529509784665), ('정당이었지', 0.41392529509784665)]
[('아직도', 0.447213595499958), ('조국', 0.447213595499958), ('찬양하는', 0.447213595499958), ('민주당', 0.447213595499958), ('한심하다', 0.447213595499958)]
[('이영손정민', 0.3824560756607918), ('수사부진', 0.3824560756607918), ('검경수사권', 0.3498058048987754), ('때문국회', 0.3824560756607918), ('상임위때', 0.3824560756607918), ('짚고', 0.3824560756607918), ('넘어', 0.3824560756607918)]
[('그', 0.0), ('그림체그', 0.7071067811865476), ('손모양머리가', 0.7071067811865476)]
[('올렸다가', 0.5773502691896257), ('삭제한', 0.5773502691896257), ('페북글', 0.5773502691896257)]
[('준석이가', 0.408248290463863), ('당대표', 0.408248290463863), ('되나요', 0.408248290463863), (

In [25]:
#TF-IDF 
#전체 CSV 파일 직접 입력 시

article_data = pd.read_csv('루리웹_엑셀확인용.csv', encoding='utf-8', header= None)
values = [' '.join(i[0].split(' ')[1:]) for i in article_data.values]

as_one = ''
for value in values:
    as_one = as_one + ' ' + value
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

#단어 빈도 정리
word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

#딕셔너리로 정리
idx2word = {ii: word for ii, word in enumerate(vocab)}

#tf-idf 
tfidf = TfidfVectorizer(max_features = 10, max_df=0.95, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(values)  #size D x V

#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)

['그알', '근황', '느낀', '민주당이', '보고', '사건', '사건이', '친구', '한강', '한강사건']


In [26]:
#TF-IDF 이미 한번 명사 단위로 정리 한 명단으로 처리할 시

as_one = ''
for noun in nouns:
    as_one = as_one + ' ' + noun
words = as_one.split()

counts = Counter(words)

vocab = sorted(counts, key=counts.get, reverse=True)

#단어 빈도 정리
word2idx = {word.encode("utf8").decode("utf8"): ii for ii, word in enumerate(vocab,1)}

#딕셔너리로 정리
idx2word = {ii: word for ii, word in enumerate(vocab)}

In [27]:
#tf-idf 
tfidf = TfidfVectorizer(max_features = 10, max_df=0.95, min_df=0)

#generate tf-idf term-document matrix
A_tfidf_sp = tfidf.fit_transform(nouns)  #size D x V

In [135]:
#tf-idf dictionary    
tfidf_dict = tfidf.get_feature_names()
print(tfidf_dict)
keyword = ",".join(tfidf_dict)
print(keyword[20:22])

['매수', '민주당', '부모', '사건', '새끼', '유튜브', '의대', '자식', '친구', '한강']
의대


In [136]:
# 감성분석

review_data = pd.read_csv('루리웹_엑셀확인용.csv',encoding='utf-8')
#print(review_data)
review_data_top = review_data[(review_data['제목'].str.contains(keyword[20:22], na=False)) | (review_data['본문'].str.contains(keyword[20:22], na=False)) | (review_data['댓글'].str.contains(keyword[20:22], na=False))]
review_data_top.head()
#print(review_data_top)

df=pd.DataFrame(review_data_top)
df.to_csv('의대.csv', index=False, encoding="utf-8-sig")
